# T6. WALL RECONSTRUCTION

In this script, we reconstruct parametric wall geometries from the instance segmentation and reference heights.
Specifically, we need:
 - T2: instances of walls, ceilings and other objects
 - T5: reference levels

## LIBRARIES

In [1]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

#import utils
import context 
import utils as utl
import utils.t6_utils as t6


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [58]:
#paths
path=Path(os.getcwd()).parents[2] # with MB this is 2

input_folder_t4=path/'data'/'t4'/'test' 
input_folder_t5=path/'data'/'t5'/'test' 

class_file=path/'data'/'_classes.json'
output_folder=path/'data'/'t6'/'test'
os.makedirs(output_folder, exist_ok=True)

#thresholds
t_level=0.5
t_inliers=0.7 # % inliers for a wall to consider its own members or the neirby ceilings and floors for its position
t_distance=0.7
t_thickness=0.127
t_topology=5
t_intersection=0.5
t_orthogonal=0.5
t_topo_floors_ceilings=False

Import Classes

In [5]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


import Levels

In [6]:
# graph=Graph().parse(graphPath)
# nodes=tl.graph_to_nodes(graph)
# wallNodes=[n for n in nodes if 'Walls' in n.subject and type(n)==PointCloudNode]
# ceilingsNodes=[n for n in nodes if 'Ceilings' in n.subject and type(n)==PointCloudNode]
# floorsNodes=[n for n in nodes if 'Floors' in n.subject and type(n)==PointCloudNode]
# print(f'{len(wallNodes)} wallNodes detected!')
# print(f'{len(ceilingsNodes)} ceilingsNodes detected!')
# print(f'{len(floorsNodes)} floorsNodes detected!')

In [7]:
# graphfiles=utl.get_list_of_files(input_folder_t5,'.ttl')
# for f in graphfiles[:1]: #only read the first one
#     print(f'processing {ut.get_filename(f)}...')      
#     levelNodes=tl.graph_path_to_nodes(f)
#     for n in levelNodes:
#         n.resource=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(n.orientedBounds)))

#     print(f'{len(levelNodes)} levelNodes detected!')
    

Import PCD

In [69]:
point_cloud_files=utl.get_list_of_files(input_folder_t4,'.laz')
level_files=utl.get_list_of_files(input_folder_t5,'.ttl')

for f_pcd,f_rdf in zip(point_cloud_files[-1:],level_files[-1:]): #only read the first one
    
    #read point cloud nodes
    pcdNodes=utl.match_graph_with_las(f_pcd,class_dict,getResources=True,getNormals=True)
    
    #read levelNodes
    levelNodes=tl.graph_path_to_nodes(f_rdf)
    for n in levelNodes:
        n.resource=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(n.orientedBounds)))

    #get wallNodes
    wallNodes=[n for n in pcdNodes if n.class_id==2]
    #get ceiling and floor nodes
    referenceNodes=[n for n in pcdNodes if n.class_id in[0,1]]
    print(f'{len(wallNodes)} pcdNodes and {len(levelNodes)} levels found')  
    
    #estimate parameters
    print('Estimating base_constraint...')
    t6.compute_base_constraint(wallNodes,levelNodes,threshold_level_height=t_level)
    print('Estimating top_constraint...')
    t6.compute_top_constraint(wallNodes,levelNodes,threshold_level_height=t_level)
    print('Estimating wall_orientation...')
    t6.compute_wall_orientation(wallNodes,referenceNodes,t_thickness=t_thickness,t_distance=t_distance,t_inliers=t_inliers) # lots of spawn errors on wrong side due to error points -> do something with planes
    print('Estimating wall_thickness...')
    t6.compute_wall_thickness(wallNodes,t_thickness=t_thickness,t_distance=t_distance)
    print('Estimating wall_axis...')
    t6.compute_wall_axis(wallNodes)
    print('Estimating wall_topology...')
    # t6.compute_wall_topology(wallNodes,levelNodes,threshold_topology=t_topology)
    print('Estimating wall_geometry...')
    t6.compute_wall_geometry(wallNodes)
    
    #write this information to the 3D detection json
    json_data=t6.walls_to_json(wallNodes,f_pcd)
    with open(os.path.join(output_folder,f'{ut.get_filename(f_pcd)}_walls.json'), 'w') as file:
        json.dump(json_data, file, indent=4)
    print("JSON data written to file:", os.path.join(output_folder,f'{ut.get_filename(f_pcd)}_walls.json') )
    
    
    #write the walls to obj
    utl.write_obj_with_submeshes(os.path.join(output_folder,f'{ut.get_filename(f_pcd)}_walls.obj') , [n.wall for n in wallNodes], [n.name for n in wallNodes])
    
    #create BIMNodes
    wallNodesBIM=[]
    for i,n in enumerate(wallNodes):
        b=BIMNode(subject=n.subject,
                  derivedFrom=pcdNodes[i].subject,
                resource=n.wall,
                base_constraint=n.base_constraint.subject,
                base_constraint_name=n.base_constraint.name,
                base_offset=n.base_offset,
                top_constraint=n.top_constraint.subject,
                top_constraint_name=n.top_constraint.name,
                top_offset=n.top_offset,
                height=n.height,
                wallThickness=n.wallThickness,
                wallLength=n.wallLength,
                normal=n.normal,
                startpoint=n.boundaryPoints[0],
                endpoint=n.boundaryPoints[1],
                color=n.color)
        wallNodesBIM.append(b)        
    graphPath=    os.path.join(output_folder,f'{ut.get_filename(f_pcd)}_walls.ttl')
    new_graph=tl.nodes_to_graph(wallNodesBIM,graphPath=graphPath,save=True)

processing 34_Parking_04_F1_small_pred...
Function match_graph_with_las took 4.1176 seconds to execute.
24 pcdNodes and 3 levels found
Estimating base_constraint...
name: 34_Parking_04_F1_small_pred_walls_201, base_constraint: level_00, base_offset: 0.04468099
name: 34_Parking_04_F1_small_pred_walls_202, base_constraint: level_00, base_offset: 0.02468099
name: 34_Parking_04_F1_small_pred_walls_203, base_constraint: level_00, base_offset: 0.02468099
name: 34_Parking_04_F1_small_pred_walls_204, base_constraint: level_00, base_offset: 2.09615099
name: 34_Parking_04_F1_small_pred_walls_205, base_constraint: level_00, base_offset: 1.05758099
name: 34_Parking_04_F1_small_pred_walls_206, base_constraint: level_00, base_offset: 3.2946809900000003
name: 34_Parking_04_F1_small_pred_walls_208, base_constraint: level_00, base_offset: 0.03468099
name: 34_Parking_04_F1_small_pred_walls_210, base_constraint: level_00, base_offset: 0.03468099
name: 34_Parking_04_F1_small_pred_walls_211, base_constrain

In [ ]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in wallNodes if n.resource is not None])
o3d.visualization.draw_geometries([joined_pcd]+[n.resource for n in levelNodes])

In [ ]:
# o3d.visualization.draw_geometries([joined_pcd]+
#                                   [n.orientedBoundingBox for n in wallNodes]+
#                                   [n.axis for n in wallNodes]+
#                                   [o3d.geometry.PointCloud(o3d.utility.Vector3dVector(n.boundaryPoints)) for n in wallNodes])

In [70]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in wallNodes if n.resource is not None])
# sampled_ceilings_and_floors=gmu.sample_geometry(gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in pcdNodes if n.class_id in[0,1]]))[0]
for n in wallNodes:
    if n.flipped:
        n.wallBox.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([joined_pcd]+
                                #   [sampled_ceilings_and_floors]+
                                  [n.wallBox for n in wallNodes]+
                                  [n.axis for n in wallNodes]+
                                  [o3d.geometry.PointCloud(o3d.utility.Vector3dVector(n.boundaryPoints)) for n in wallNodes])

## PROCESSING

Compute base constraint

In [ ]:
# wallNodes=[n for n in pcdNodes if n.class_id==2]
# for n in wallNodes:
#     #compute minheight of the resource at 0.1% of the height (absolute minimum might be wrong)
#     z_values = np.sort(np.asarray(n.resource.points)[:,2])
#     minheight = np.percentile(z_values, 0.1)

#     #compute base constraint. select the intersecting level that is closest to the bottom of the wall. Else, just take first levelNode.
#     nearby_ref_levels= tl.select_nodes_with_intersecting_bounding_box(n,levelNodes)
#     n.base_constraint= next((n for n in nearby_ref_levels if np.absolute(n.height-minheight)<t_level),levelNodes[0])  if nearby_ref_levels else levelNodes[0] #this is a link!
    
#     #compute base offset
#     n.base_offset=minheight-n.base_constraint.height
#     print(f'name: {n.name}, base_constraint: {n.base_constraint.name}, base_offset: {n.base_offset}')


name: Walls_838, base_constraint: level_00, base_offset: 0.29026122868419274
name: Walls_1571, base_constraint: level_00, base_offset: 0.1012612286841927
name: Walls_9891, base_constraint: level_00, base_offset: 0.22026022868419248
name: Walls_10086, base_constraint: level_00, base_offset: 0.0752612286841929
name: Walls_10150, base_constraint: level_00, base_offset: 1.0851852286841928
name: Walls_10253, base_constraint: level_00, base_offset: 0.8348252286841926
name: Walls_10449, base_constraint: level_00, base_offset: 0.1352592286841924
name: Walls_10619, base_constraint: level_00, base_offset: 0.08626122868419213
name: Walls_10734, base_constraint: level_00, base_offset: 0.20115722868419242
name: Walls_12553, base_constraint: level_00, base_offset: -0.08497277131580712
name: Walls_12607, base_constraint: level_00, base_offset: -0.08667177131580783
name: Walls_12651, base_constraint: level_00, base_offset: -0.08173877131580713
name: Walls_12688, base_constraint: level_00, base_offset:

In [ ]:
# o3d.visualization.draw_geometries([n.resource,o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(n.orientedBoundingBox)]+[levelNodes[0].resource])

Compute top constraint

In [ ]:
# for n in wallNodes:
#     #compute maxheight of the resource at 0.1% of the height (absolute minimum might be wrong)
#     z_values = np.sort(np.asarray(n.resource.points)[:,2])
#     minheight = np.percentile(z_values, 0.1)
#     maxheight = np.percentile(z_values, 99.9)

#     #compute base constraint. select the intersecting level that is closest to the top of the wall. Else, just take last levelNode.
#     nearby_ref_levels= tl.select_nodes_with_intersecting_bounding_box(n,levelNodes)
#     n.top_constraint= next((n for n in nearby_ref_levels if np.absolute(n.height-maxheight)<t_level),levelNodes[-1]) if nearby_ref_levels else levelNodes[-1] #this is a link!
    
#     #compute base offset
#     n.top_offset=maxheight-n.top_constraint.height

#     #compute wall height
#     n.height=maxheight-minheight
#     print(f'name: {n.name}, top_constraint: {n.top_constraint.name}, top_offset: {n.top_offset}')


name: Walls_838, top_constraint: level_10, top_offset: 0.058121983037024094
name: Walls_1571, top_constraint: level_10, top_offset: 0.06512198303702377
name: Walls_9891, top_constraint: level_10, top_offset: 0.07712298303702525
name: Walls_10086, top_constraint: level_10, top_offset: 0.05712198303702465
name: Walls_10150, top_constraint: level_10, top_offset: 0.07465998303702515
name: Walls_10253, top_constraint: level_10, top_offset: 0.06312198303702488
name: Walls_10449, top_constraint: level_10, top_offset: 0.04812198303702431
name: Walls_10619, top_constraint: level_10, top_offset: 0.054121983037024535
name: Walls_10734, top_constraint: level_10, top_offset: 0.004121983037023824
name: Walls_12553, top_constraint: level_10, top_offset: -0.016878016962975195
name: Walls_12607, top_constraint: level_10, top_offset: 0.12792098303702693
name: Walls_12651, top_constraint: level_10, top_offset: 0.16712198303702408
name: Walls_12688, top_constraint: level_10, top_offset: 0.1629029830370272

In [ ]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in wallNodes if n.resource is not None])
# for n in wallNodes:
#     n.orientedBoundingBox.color=[1,0,0]
# o3d.visualization.draw_geometries([joined_pcd]+[n.orientedBoundingBox for n in wallNodes])

Compute Wall Orientation

In [ ]:
# for n in wallNodes:    
#     #Compute the dominant plane on the point cloud
#     n.plane_model, inliers = n.resource.segment_plane(distance_threshold=0.03,
#                                             ransac_n=3,
#                                             num_iterations=1000)
    
#     #get center of the face and postion it on the correct height (base constraint + base offset)   
#     n.faceCenter=n.resource.select_by_index(inliers).get_center()  
#     n.faceCenter[2]=n.base_constraint.height + n.base_offset

#     #compute the normal of the plane in 2D (third component should be zero, normal should point outwards of the wall)
#     n.normal=n.plane_model[:3]
#     n.normal[2]=0
#     n.normal/=np.linalg.norm(n.normal)


#     #compute the sign.
#     #if n.orientedBoundingBox width is > than 0.1, the sign is  given by the dotproduct of the normal of the face with the vector between the center of the face and the center of the oriented bounding box
#     boxCenter=n.orientedBoundingBox.get_center()
#     boxCenter[2]=n.base_constraint.height + n.base_offset
#     n.sign=np.sign(np.dot(n.normal,n.faceCenter-boxCenter))

#     #if n.orientedBoundingBox width < t_thickness, take a look at the ceiling and floor nodes to see on which side they are, and use them to spawn the wall away from these nodes
#     if n.orientedBoundingBox.extent[2]<=t_thickness:   
#         #combine list
#         combined_list = [n for n in pcdNodes if n.class_id in [0,1]]
#         #create reference pcd from these resources
#         referencePcd,identityArray=gmu.create_identity_point_cloud([n.resource for n in combined_list if n.resource is not None])
#         #find nearest point near the top and the bottom 
#         topPoint=copy.deepcopy(boxCenter)
#         topPoint[2]=n.base_constraint.height + n.base_offset+n.height
#         bottomPoint=boxCenter
#         idx,distances=gmu.compute_nearest_neighbors(np.asarray([topPoint,bottomPoint]),np.asarray(referencePcd.points)) 
#         points=np.asarray(referencePcd.points)[idx[:,0]]
#         #compute orthogonal distance to the plane and select node with lowest distance
#         idx=idx[np.argmin(np.absolute(np.einsum('i,ji->j',n.normal,points-boxCenter))) ][0]
#         index=identityArray[idx]
#         referenceNode=combined_list[index]
#         point=np.asarray(referencePcd.points)[idx]
#         point[2]=n.base_constraint.height + n.base_offset
#         n.sign=np.sign(np.dot(n.normal,point-boxCenter))
    
#     #flip the normal if it points inwards
#     n.normal*=-1 if n.sign==-1 else 1

#     print(f'name: {n.name}, plane: {n.plane_model}, inliers: {len(inliers)}/{len(np.asarray(n.resource.points))}')      


name: Walls_838, plane: [-0.01528822 -0.99988313 -0.         12.1552986 ], inliers: 2242/2633
name: Walls_1571, plane: [9.99999959e-01 2.85740211e-04 0.00000000e+00 2.06578808e+01], inliers: 57356/75779
name: Walls_9891, plane: [ 9.99995606e-01 -2.96449008e-03  0.00000000e+00  2.03282177e+01], inliers: 4167/6000
name: Walls_10086, plane: [ 9.99976459e-01 -6.86154984e-03  0.00000000e+00  2.00000192e+01], inliers: 9919/12508
name: Walls_10150, plane: [ 0.04715889 -0.9988874  -0.         11.6405077 ], inliers: 1212/1463
name: Walls_10253, plane: [-9.99943286e-01  1.06500808e-02 -0.00000000e+00 -2.36273864e+01], inliers: 1240/1783
name: Walls_10449, plane: [0.04617253 0.99893348 0.         5.61366348], inliers: 1179/1615
name: Walls_10619, plane: [-1.77134186e-02  9.99843105e-01  0.00000000e+00  2.64397845e+01], inliers: 9491/12447
name: Walls_10734, plane: [-7.35930033e-03  9.99972920e-01  0.00000000e+00  2.67450647e+01], inliers: 6487/7897
name: Walls_12553, plane: [-9.99988815e-01  4.72

Compute Wall thickness (this seems very subjective)

In [ ]:
# for n in wallNodes:
#     #compute the normals of the wall
#     pcd_tree = o3d.geometry.KDTreeFlann(n.resource)
#     n.resource.estimate_normals() if not n.resource.has_normals() else None

#     #for every 100th point in P, that has the same normal as the dominant plane, select nearest points in P that meet a distance threshold    
#     points=np.asarray(n.resource.points)[::100]
#     normals=np.asarray(n.resource.normals)[::100]
#     idx=np.where(np.absolute(np.einsum('i,ji->j',n.normal,normals))>0.9)
#     points=points[idx]
#     normals=normals[idx]
#     distances=[]

#     for p,q in zip(points,normals):
#         #compute distances
#         [k, idx, _] = pcd_tree.search_radius_vector_3d(p, t_distance)        
#         #retain only the distances for which the normal is within 0.7 radians of the normal of the point
#         kNormals=np.asarray(n.resource.normals)[idx]
#         indices=np.asarray(idx)[np.where(np.absolute(np.einsum('i,ji->j',q,kNormals))>0.9)]
#         #compute the dotproduct between the point and the normals of the points in the radius
#         vectors=p-np.asarray(n.resource.select_by_index(indices).points)
#         #keep the max distance (orthogonal distance between the planes)
#         distances.append(np.absolute(np.einsum('i,ji->j', q, vectors)).max())

#     #take the distance at the 99% percentile
#     distance = np.percentile(np.sort(np.array(distances)), 90) 
#     n.wallThickness=distance if distance > t_thickness else t_thickness

#     print(f'name: {n.name}, BB_extent: {n.orientedBoundingBox.extent}, wallThickness: {n.wallThickness}')


name: Walls_838, BB_extent: [2.56829854 0.6480327  0.16037955], wallThickness: 0.1556695625636702
name: Walls_1571, BB_extent: [15.06414852  2.97204777  0.45455243], wallThickness: 0.256799986085673
name: Walls_9891, BB_extent: [2.80539658 1.01270262 0.52784626], wallThickness: 0.3074222266679428
name: Walls_10086, BB_extent: [2.770266   0.90024241 0.44330392], wallThickness: 0.12230053312956729
name: Walls_10150, BB_extent: [1.77913083 0.86544186 0.2425967 ], wallThickness: 0.12
name: Walls_10253, BB_extent: [2.05493523 0.89227846 0.41905656], wallThickness: 0.3419580185169591
name: Walls_10449, BB_extent: [2.73363879 0.48133087 0.24654717], wallThickness: 0.1655250744468178
name: Walls_10619, BB_extent: [2.76352239 1.02352871 0.44998086], wallThickness: 0.3121825666465795
name: Walls_10734, BB_extent: [2.62850902 0.98396136 0.46797811], wallThickness: 0.3510181697572535
name: Walls_12553, BB_extent: [4.89898941 2.96053728 0.47667371], wallThickness: 0.30284778554026187
name: Walls_12

In [ ]:
# o3d.visualization.draw_geometries([n.resource for n in wallNodes if n.wallThickness <=t_thickness]+
#                                   [n.orientedBoundingBox for n in wallNodes])

Compute Wall axis

In [ ]:
# for n in wallNodes:     
    
#     #offset the center of the plane with half the wall thickness in the direction of the normal of the plane  
#     # wallCenter=n.faceCenter-n.sign*n.normal*n.wallThickness/2 
#     wallCenter=n.faceCenter-n.normal*n.wallThickness/2 

#     wallCenter[2]=n.base_constraint.height + n.base_offset

#     #project axis aligned bounding points on the plane
#     box=n.resource.get_axis_aligned_bounding_box()    
#     points=np.asarray(box.get_box_points())
#     points[:,2]=n.base_constraint.height + n.base_offset

#     #translate the points to the plane
#     vectors=points-wallCenter
#     translation=np.einsum('ij,j->i',vectors,n.normal)
#     points=points - translation[:, np.newaxis] * n.normal

#     # Calculate the pairwise distances between all boundary points
#     distances = np.linalg.norm(points[:, np.newaxis] - points, axis=2)

#     # Get the indices of the two points with the maximum distance
#     max_indices = np.unravel_index(np.argmax(distances), distances.shape)

#     # Retain only the two points with the maximum distance
#     n.boundaryPoints = points[max_indices,:]

#     #create the axis
#     n.axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(n.boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1])
#     n.startPoint=n.boundaryPoints[0]
#     n.endPoint=n.boundaryPoints[1]
#     # Calculate the length
#     n.wallLength = np.linalg.norm(n.boundaryPoints[0] - n.boundaryPoints[1])

#     print(f'name: {n.name}, wallLength: {n.wallLength}')


name: Walls_838, wallLength: 0.6444628124468508
name: Walls_1571, wallLength: 15.092114537193124
name: Walls_9891, wallLength: 0.9654009323743882
name: Walls_10086, wallLength: 0.8909638700296282
name: Walls_10150, wallLength: 0.35446018177138294
name: Walls_10253, wallLength: 0.8951669000688076
name: Walls_10449, wallLength: 0.49349079441010407
name: Walls_10619, wallLength: 0.9832333661168238
name: Walls_10734, wallLength: 0.9839318732367888
name: Walls_12553, wallLength: 4.8489653225556495
name: Walls_12607, wallLength: 6.980302895175979
name: Walls_12651, wallLength: 4.846087215826779
name: Walls_12688, wallLength: 6.9770426417893505
name: Walls_39563, wallLength: 43.98602545552052
name: Walls_39816, wallLength: 6.865011163167261
name: Walls_39979, wallLength: 4.886394835587181
name: Walls_40097, wallLength: 10.566424214940604
name: Walls_40301, wallLength: 3.6875897286258947
name: Walls_40356, wallLength: 3.791710093399384
name: Walls_40452, wallLength: 4.384815719012658
name: Wal

In [ ]:
# o3d.visualization.draw_geometries([joined_pcd]+
#                                   [n.orientedBoundingBox for n in wallNodes]+
#                                   [n.axis for n in wallNodes]+
#                                   [o3d.geometry.PointCloud(o3d.utility.Vector3dVector(n.boundaryPoints)) for n in wallNodes])

Compute Wall Geometry

In [ ]:
# for n in wallNodes:
#     pointList=[]
#     points=np.asarray(n.axis.points)
#     # pointList.extend(points+n.sign*n.normal*n.wallThickness/2)
#     pointList.extend(points+n.normal*n.wallThickness/2)

#     # pointList.extend(points-n.sign*n.normal*n.wallThickness/2)
#     pointList.extend(points-n.normal*n.wallThickness/2)

#     pointList.extend(np.array(pointList)+np.array([0,0,n.height]))
#     pcd=o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(pointList))

#     box=pcd.get_oriented_bounding_box()
#     n.wall=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(box)
#     n.wall.paint_uniform_color(ut.literal_to_array(n.color))
#     n.wallBox=o3d.geometry.LineSet.create_from_oriented_bounding_box(box)
#     n.wallBox.paint_uniform_color([0,0,1])

#     print(f'name: {n.name}, wall: {n.wall}')


name: Walls_838, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_1571, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_9891, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_10086, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_10150, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_10253, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_10449, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_10619, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_10734, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_12553, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_12607, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_12651, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_12688, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_39563, wall: TriangleMesh with 8 points and 12 triangles.
name: Walls_39816, wall:

In [ ]:
# o3d.visualization.draw_geometries([joined_pcd]+
#                                   [n.wallBox for n in wallNodes]+
#                                   [n.axis for n in wallNodes]+
#                                   [o3d.geometry.PointCloud(o3d.utility.Vector3dVector(n.boundaryPoints)) for n in wallNodes])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 


Compute Wall topology

In [ ]:
# def intersect_line_2d(p0, p1, q0, q1,strict=True):
#     """
#     Compute the intersection between two lines in 3D.
#     Each line is defined by a pair of points.
    
#     Parameters:
#     - p0, p1: points (arrays) defining the first line.
#     - q0, q1: points (arrays) defining the second line.
#     - strict: if True, the intersection point must lie within the line segments.
    
#     Returns:
#     - The intersection point as a numpy array if it exists, otherwise None.
#     """
#     # Direction vectors of the lines
#     dp = p1 - p0
#     dq = q1 - q0
    
#     # Matrix and vector for the linear system
#     A = np.vstack((dp, -dq)).T
#     b = q0 - p0
    
#     # Solve the linear system
#     try:
#         t, u = np.linalg.solve(A, b)
#     except np.linalg.LinAlgError:
#         # The system is singular: lines are parallel or identical
#         return None
    
#     # Intersection point
#     intersection = p0 + t * dp
    
#     if strict:
#     # Since the system has a solution, check if it lies within the line segments
#         if np.allclose(intersection, q0 + u * dq):
#             return intersection
#         else:
#             return None
#     else:
#         return intersection


In [ ]:
# from scipy.spatial import distance

# #in preparation, compute orthonal curves to the wall axes at the start and end point
# for n in wallNodesBIM:
#     n.orthogonalStartpoint = n.startPoint + n.normal * n.wallThickness/2
#     n.orthogonalEndpoint = n.endPoint + n.normal * n.wallThickness/2     

# #next, investigate the connections between the walls
# for n in wallNodesBIM:
#     #gather the start and end point the nearby walls
#     axisPoints=np.array([n.startPoint,n.endPoint])
#     axisPointsOthogonal=np.array([n.orthogonalStartpoint,n.orthogonalEndpoint])

#     #create reference point cloud of all wall start-and endpoints
#     referencePcd,identityArray=gmu.create_identity_point_cloud([w.resource for w in wallNodesBIM if w!=n])

#     #compute nearest neighbors
#     idx,distances=gmu.compute_nearest_neighbors(axisPoints,referencePcd,n=10)
#     #filter out the points that are too far away
#     idx=idx[np.where(distances<t_intersection)]
#     points=np.asarray(referencePcd.points)[idx]
#     indices=identityArray[idx]
#     nearbyWalls=[wallNodes[i] for i in indices]
#     print(f'found {len(nearbyWalls)} nearby walls for {n.name}')

#     #1.compute the intersections between neighboring curves
#     for w in nearbyWalls:
#         intersection_point = intersect_line_2d(n.startPoint, n.endPoint, w.startPoint, w.endPoint,strict=False)
#         if intersection_point is not None:
#             #compute the distance between the intersection point and the wall axes
#             d = distance.euclidean(intersection_point, n.startPoint)
#             #filter out the points that are too far away
#             if d<t_intersection:
#                 print(f'intersection between {n.name} and {w.name} at {intersection_point}, distance: {d}',strict=False)
                
#     #2.compute a orthogonal curve to the wall axis at the start and end point
#     for w in nearbyWalls:
#         intersection_point = intersect_line_2d(n.orthogonalStartpoint, n.orthogonalEndpoint, w.startPoint, w.endPoint)
#         if intersection_point is not None:
#             #compute the distance between the intersection point and the wall axes
#             d = distance.euclidean(intersection_point, n.orthogonalStartpoint)
#             #filter out the points that are too far away
#             if d<t_orthogonal:
#                 print(f'orthogonal intersection between {n.name} and {w.name} at {intersection_point}, distance: {d}')
                

#     break


ValueError: Expected 2D array, got scalar array instead:
array=PointCloud with 128620 points..
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## EXPORT

json with metadata

In [ ]:
# #declare json
# json_data = {
#         "filename": ut.get_filename(os.path.join(output_folder,f'{ut.get_filename(f)}_walls.obj') ), 
#         "objects": []
#     }
# #fill json
# for n in wallNodes:
#     obj = {
#             "name": n.name,
#             "base_constraint":n.base_constraint.name,
#             "base_offset":n.base_offset,
#             "top_constraint":n.top_constraint.name,
#             "top_offset":n.top_offset,
#             "height": n.height,
#             "wallThickness": n.wallThickness,
#             "wallLength": n.wallLength,
#             "normal": {
#                 "x": n.normal[0],
#                 "y": n.normal[1],
#                 "z": n.normal[2]
#             },
#             "startpoint": {
#                 "x": n.boundaryPoints[0][0],
#                 "y": n.boundaryPoints[0][1],
#                 "z": n.boundaryPoints[0][2]
#             }
#             ,
#             "endpoint": {
#                 "x": n.boundaryPoints[1][0],
#                 "y": n.boundaryPoints[1][1],
#                 "z": n.boundaryPoints[1][2]
#             }
#             }
#     json_data["objects"].append(obj)
# #write this information to the 3D detection json
# with open(os.path.join(output_folder,f'{ut.get_filename(f)}_walls.json') , "w") as json_file: 
#     json.dump(json_data, json_file, indent=4)
# print("JSON data written to file:", os.path.join(output_folder,f'{ut.get_filename(f)}_walls.json') )

JSON data written to file: /home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t6/test/05_MedOffice_01_F2_small1_walls.json


obj with walls

In [ ]:
# def write_obj_with_submeshes(filename, meshes, mesh_names):
#     """
#     Write multiple Open3D TriangleMesh objects to a single OBJ file with submeshes.

#     Parameters:
#     - filename: str, the name of the output OBJ file.
#     - meshes: list of open3d.geometry.TriangleMesh, the meshes to write.
#     - mesh_names: list of str, the names of the submeshes.
#     """
#     if len(meshes) != len(mesh_names):
#         raise ValueError("meshes and mesh_names must have the same length")

#     vertex_offset = 1  # OBJ files are 1-indexed
#     with open(filename, 'w') as file:
#         for mesh, name in zip(meshes, mesh_names):
#             file.write(f"g {name}\n")  # Start a new group for the submesh

#             # Write vertices
#             for vertex in mesh.vertices:
#                 file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")

#             # Write faces, adjusting indices based on the current offset
#             for triangle in mesh.triangles:
#                 adjusted_triangle = triangle + vertex_offset
#                 file.write(f"f {adjusted_triangle[0]} {adjusted_triangle[1]} {adjusted_triangle[2]}\n")

#             # Update the vertex offset for the next mesh
#             vertex_offset += len(mesh.vertices)

# # Example usage:
# # Assuming mesh1 and mesh2 are your Open3D TriangleMesh objects
# mesh1 = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
# mesh1.compute_vertex_normals()

# mesh2 = o3d.geometry.TriangleMesh.create_box(width=1.0, height=1.0, depth=1.0)
# mesh2.compute_vertex_normals()

# write_obj_with_submeshes(os.path.join(output_folder,f'{ut.get_filename(f)}_walls.obj') , [n.wall for n in wallNodes], [n.name for n in wallNodes])

graph with metadata

In [ ]:
# wallNodesBIM=[]
# for n in wallNodes:
#     b=BIMNode(subject=n.subject+'_BIM',
#             derivedFrom=n.subject, #this should be a URI
#             resource=n.wall,
#             base_constraint=n.base_constraint.subject,
#             base_constraint_name=n.base_constraint.name,
#             base_offset=n.base_offset,
#             top_constraint=n.top_constraint.subject,
#             top_constraint_name=n.top_constraint.name,
#             top_offset=n.top_offset,
#             height=n.height,
#             wallThickness=n.wallThickness,
#             wallLength=n.wallLength,
#             normal=n.normal,
#             startpoint=n.boundaryPoints[0],
#             endpoint=n.boundaryPoints[1],
#             color=n.color)
#     wallNodesBIM.append(b)
    
# graphPath=    os.path.join(output_folder,f'{ut.get_filename(f)}_walls.ttl')
# new_graph=tl.nodes_to_graph(wallNodesBIM,graphPath=graphPath,save=True)

# #remove all BIM nodes from original graph
# for n in wallNodesBIM:
#     graph.remove((URIRef(n.subject),None,None))
# graph=graph+new_graph
# print(graph.serialize(graphPath, format="turtle"))